In [6]:
import sys
import os

import numpy as np 

import matplotlib.pyplot as plt

from src.model import *
from src.dataset import MyDataset
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm
from IPython.display import clear_output

from src.utils import (
    set_seed,
    root_dir,
    save_ckpt,
    load_ckpt,
    log_meters,
    reset_meters,
    create_meters,
    print_cfg_info,
    load_data_scaler,
)

In [2]:
cfg = {
    "lbl": "train_4_standard_best",
    "desc": "predicts point of size 4 from standard scaled train data",
    "data_suffix": "_standard",
    "z_dim": 256,
    "gen_dims": [128, 32, 8, 4],
    "disc_dims": [4, 128, 32, 8],
    "num_epochs": 1000,
    "betas": [0, 0.99],
    "lr": 1e-3,
    "batch_size": 250,
    "w_gp": 10,
    "w_eps": 1e-3,
    "w_weak":1e-3,
    "display_freq": 10,
    "metrics_freq": 10,
    "seed": 5051,
    "device": torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"),
    "num_pred_rows": 410,
    "num_reps": 10000,
    "finetune": True,
    "from_ckpt": True,
    "from_epoch": 2
}

cgf_model = {
    
}

cfg_dataset_weak = {
    'n_samples_train':250,
    'n_samples_test':250,
    'n_samples_weak':750,
}

cfg_dataset = {
    'n_samples_train':250,
    'n_samples_test':250,
}

cfg_make_class = {
    'n_samples':cfg_dataset['n_samples_train']\
                +cfg_dataset['n_samples_test'],
    'n_features':50,
    'n_informative':10,
    'n_classes':5,
    'shuffle':True
}

cfg_loader = {  
    "batch_size":cfg["batch_size"],
    "drop_last":False,
}

In [3]:
X_train, X_test, y_train, y_test = train_test_split(*make_classification(**cfg_make_class),
                                                    test_size=cfg_dataset["n_samples_test"])
train_loader = MyDataset(X_train, y_train, batch_size=cfg["batch_size"])
test_loader = MyDataset(X_test, y_test, batch_size=cfg["batch_size"])

In [ ]:
netG = Generator(cfg_model)
netD = Discriminator(cfg_model)

cfg_opt = {
    'betas':[0, 0.99],
    'lr':1e-3,
}

optimG = Adam(filter(lambda p: p.requires_grad, netG.parameters()), **cfg_opt)
optimD = Adam(filter(lambda p: p.requires_grad, netD.parameters()), **cfg_opt)

meter_names = ['netG_weak_loss','eps_loss', 'netD_loss', 'netG_loss', 'grad_loss', 'netD_real_loss', 'netD_fake_loss']
meters = create_meters(meter_names)